# Simple Convnet - MNIST

Slightly modified from mnist_cnn.py in the Keras examples folder:

https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [1]:
WEIGHTS_FILEPATH = 'mnist_cnn.hdf5'
MODEL_ARCH_FILEPATH = 'mnist_cnn.json'

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [3]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# Sequential Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Model saving callback
checkpointer = ModelCheckpoint(filepath=WEIGHTS_FILEPATH, monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

# Train
batch_size = 128
epochs = 100
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2,
          callbacks=[checkpointer, early_stopping], 
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.98000, saving model to mnist_cnn.hdf5
6s - loss: 0.2856 - acc: 0.9132 - val_loss: 0.0613 - val_acc: 0.9800
Epoch 2/100
Epoch 00001: val_acc improved from 0.98000 to 0.98770, saving model to mnist_cnn.hdf5
4s - loss: 0.1004 - acc: 0.9702 - val_loss: 0.0394 - val_acc: 0.9877
Epoch 3/100
Epoch 00002: val_acc improved from 0.98770 to 0.98850, saving model to mnist_cnn.hdf5
4s - loss: 0.0734 - acc: 0.9775 - val_loss: 0.0353 - val_acc: 0.9885
Epoch 4/100
Epoch 00003: val_acc improved from 0.98850 to 0.98940, saving model to mnist_cnn.hdf5
4s - loss: 0.0608 - acc: 0.9818 - val_loss: 0.0305 - val_acc: 0.9894
Epoch 5/100
Epoch 00004: val_acc improved from 0.98940 to 0.98990, saving model to mnist_cnn.hdf5
4s - loss: 0.0535 - acc: 0.9837 - val_loss: 0.0325 - val_acc: 0.9899
Epoch 6/100
Epoch 00005: val_acc improved from 0.98990 to 0.99170, saving model to mnist_cnn.hdf5
4s - loss: 0.0461 - 

In [6]:
with open(MODEL_ARCH_FILEPATH, 'w') as f:
    f.write(model.to_json())